In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00


In [4]:
import torch
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.datasets import MNISTSuperpixels
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch_geometric.nn import GATConv

In [5]:
train_dataset = MNISTSuperpixels(root='data/MNIST', train=True)
test_dataset = MNISTSuperpixels(root='data/MNIST', train=False)

Extracting data/MNIST/raw/MNISTSuperpixels.zip
Processing...
Done!


In [6]:
train_dataset[0]

Data(x=[75, 1], edge_index=[2, 1399], y=[1], pos=[75, 2])

In [7]:
def collate_func(original_batch: list[Data]):
    batch_node_features: list[torch.Tensor] = []
    batch_edge_indices: list[torch.Tensor] = []
    classes: list[int] = []

    for data in original_batch:
        node_features = torch.cat((data.x, data.pos), dim=-1).to(device)
        edge_indices = data.edge_index.to(device)
        class_ = int(data.y)

        batch_node_features.append(node_features)
        batch_edge_indices.append(edge_indices)
        classes.append(class_)

    collated = {
        "batch_node_features": batch_node_features,
        "batch_edge_indices": batch_edge_indices,
        "classes": torch.LongTensor(classes).to(device),
    }

    return collated

In [8]:
batch_size = 600
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,
                          collate_fn=collate_func)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_func)

In [9]:
class GNN(nn.Module):
  def __init__(
      self,
      in_channels = 3,
      hidden_dim = 152,
      num_classes = 10,
  ):
    super().__init__()

    self.in_channels = in_channels
    self.hidden_dim = hidden_dim
    self.num_classes = num_classes

    self.conv1 = GATConv(in_channels=in_channels, out_channels=hidden_dim)
    self.conv2 = GATConv(in_channels=hidden_dim, out_channels=hidden_dim)
    self.conv3 = GATConv(in_channels=hidden_dim, out_channels=hidden_dim)
    self.conv4 = GATConv(in_channels=hidden_dim, out_channels=hidden_dim)

    self.fc = nn.Sequential(
        nn.Linear(in_channels+4*hidden_dim, 256),
        nn.ReLU(True),
        nn.Linear(256, 128),
        nn.ReLU(True),
        nn.Linear(128, num_classes),
    )

  def forward_one_base(self, node_features: torch.Tensor, edge_indices: torch.Tensor) -> torch.Tensor:
    assert node_features.ndim == 2 and node_features.shape[1] == self.in_channels
    assert edge_indices.ndim == 2 and edge_indices.shape[0] == 2

    x = node_features
    x1 = self.conv1(x, edge_indices)
    x2 = self.conv2(x1, edge_indices)
    x3 = self.conv3(x2, edge_indices)
    x4 = self.conv4(x3, edge_indices)

    x_cat = torch.cat((x, x1, x2, x3, x4), dim=-1)

    x_cat = x_cat.view(x_cat.size(0), -1)
    return x_cat

  def forward(self, batch_node_features: list[torch.Tensor], batch_edge_indices: list[torch.Tensor]) -> torch.Tensor:
    assert len(batch_node_features) == len(batch_edge_indices)

    features_list = []
    for node_features, edge_indices in zip(batch_node_features, batch_edge_indices):
      features_list.append(self.forward_one_base(node_features=node_features, edge_indices=edge_indices))

    features = torch.stack(features_list, dim=0)  # BATCH_SIZE x NUM_NODES x NUM_FEATURES
    features = features.mean(dim=1)  # readout operation [BATCH_SIZE x NUM_FEATURES]

    logits = self.fc(features)
    return logits


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
model = GNN(in_channels=3, hidden_dim=64).to(device)

In [12]:
model

GNN(
  (conv1): GATConv(3, 64, heads=1)
  (conv2): GATConv(64, 64, heads=1)
  (conv3): GATConv(64, 64, heads=1)
  (conv4): GATConv(64, 64, heads=1)
  (fc): Sequential(
    (0): Linear(in_features=259, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 10

In [14]:
def train(model, loader, optimizer, criterion, batches_passed):
  model.train()
  total_loss=0
  total_correct=0
  total_samples=0

  for batch in enumerate(loader):
    batch_node_features = batch["batch_node_features"]
    batch_edge_indices = batch["batch_edge_indices"]
    classes = batch["classes"]

    logits = model(batch_node_features=batch_node_features, batch_edge_indices=batch_edge_indices)
    predicted_classes = torch.argmax(logits, dim=1)

    #loss = criterion(logits, classes).mean()
    loss = criterion(logits, classes)
    total_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #accuracy = (predicted_classes == classes).to(torch.float32).mean()
    correct = (predicted_classes == classes).sum().item()
    total_correct += correct
    total_samples += len(classes)

    '''print('Training accuracy: ',accuracy)
    print('Training loss: ',loss)
    print('Batch: ',batches_passed)'''

    batches_passed += 1

    #if i+1>num_batches:
     # break
  epoch_loss=total_loss / len(loader)
  epoch_accuracy = total_correct / total_samples

  print(f"Epoch [{batches_passed}], Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
  return batches_passed

In [15]:
def evaluate(model, loader, epochs):
  model.eval()
  accu=0.0
  samp=0.0
  total_correct=0
  total_samples=0

  for batch in loader:
    batch_node_features = batch["batch_node_features"]
    batch_edge_indices = batch["batch_edge_indices"]
    classes = batch["classes"]

    logits = model(batch_node_features=batch_node_features, batch_edge_indices=batch_edge_indices)
    predicted_classes = torch.argmax(logits, dim=1)

    '''accu += float((predicted_classes == classes).to(torch.float32).mean().cpu().numpy()) * len(classes)
    samp += len(classes)'''
    correct = (predicted_classes == classes).sum().item()
    total_correct += correct
    total_samples += len(classes)

  #accuracy = accu/samp
  epoch_accuracy = total_correct / total_samples
  print(f"Validation Accuracy after epoch [{epochs}]: {epoch_accuracy:.3f}")


  print("Accuracy: ", accu)

In [ ]:
batches_passed = 0
epochs=3

for epoc in range(epochs):
  print(f"Epoch [{epoc + 1}/{epochs}]")
  batches_passed = train(
      model=model,
      optimizer=optimizer,
      loader=train_loader,
      criterion=criterion,
      batches_passed=batches_passed,
      )

  evaluate(
      model=model,
      loader=test_loader,
      epochs=epoc + 1,
      )


Epoch [1/3]


KeyboardInterrupt: 

In [ ]:
epochs = 10
for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, criterion, 4,8)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.3f}")

Training accuracy:  tensor(0.1406)
Training loss:  tensor(2.3268, grad_fn=<MeanBackward0>)
Batch:  4
Training accuracy:  tensor(0.0625)
Training loss:  tensor(2.3515, grad_fn=<MeanBackward0>)
Batch:  5
Training accuracy:  tensor(0.0781)
Training loss:  tensor(2.3663, grad_fn=<MeanBackward0>)
Batch:  6
Training accuracy:  tensor(0.1094)
Training loss:  tensor(2.3250, grad_fn=<MeanBackward0>)
Batch:  7
Training accuracy:  tensor(0.0938)
Training loss:  tensor(2.3463, grad_fn=<MeanBackward0>)
Batch:  8
Training accuracy:  tensor(0.0312)
Training loss:  tensor(2.3540, grad_fn=<MeanBackward0>)
Batch:  9
Training accuracy:  tensor(0.1250)
Training loss:  tensor(2.2509, grad_fn=<MeanBackward0>)
Batch:  10
Training accuracy:  tensor(0.1094)
Training loss:  tensor(2.3201, grad_fn=<MeanBackward0>)
Batch:  11
Training accuracy:  tensor(0.0938)
Training loss:  tensor(2.3267, grad_fn=<MeanBackward0>)
Batch:  12
Epoch 1/10, Train Loss: 13.000
Training accuracy:  tensor(0.1562)
Training loss:  tensor

In [18]:
#hyperparameter tuning
batch_size = 4
epochs = 2
hidden_dim = 64
optimizer1 = torch.optim.Adam(model.parameters(), lr=0.0001) #lr=0.0001
train(model, train_loader, optimizer1, criterion, 3)

In [19]:
#secodn
batch_size = 8 #changed
epochs = 2
hidden_dim = 164 #changed
lr = 0.002
optimizer2 = torch.optim.Adam(model.parameters(), lr=0.002) #lr=0.002
train(model, train_loader, optimizer2, criterion, 3)

In [21]:
import networkx as nx
import matplotlib as plt


def visualize_graphs(data_loader, num_samples=5):
    for i in range(10):
        print(f"Class: {i}")
        class_indices = [idx for idx, data in enumerate(data_loader.dataset) if data.y == i]
        sample_indices = class_indices[:num_samples]
        for idx in sample_indices:
            data = data_loader.dataset[idx]
            edge_index = data.edge_index.cpu().numpy()
            G = nx.Graph()
            G.add_edges_from(edge_index.T)
            plt.figure()
            nx.draw(G, with_labels=True)
            plt.title(f"Sample {idx} from class {i}")
            plt.show()


In [23]:
visualize_graphs(test_loader)

In [24]:
#CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 32 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [25]:
model_cnn = CNNModel()

criterion_cnn = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(model.parameters(), lr=0.001)

In [29]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loaderr = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loaderr = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 103197289.60it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 80169221.59it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32205883.38it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13715283.49it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [30]:
def train_model(model, train_loader, test_loader, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):  # Modified this line
            optimizer_cnn.zero_grad()

            outputs = model(inputs)
            loss = criterion_cnn(outputs, labels)
            loss.backward()
            optimizer_cnn.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

        # Validate the model
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                inputs, labels = data
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Accuracy on test set: {100 * correct / total}%")


In [ ]:
train_model(model_cnn, train_loaderr, test_loaderr, num_epochs=3)

Epoch 1/3, Loss: 2.30235404602246
Accuracy on test set: 8.73%
